# Анализ результатов эксперимента от 23.09.25

In [ ]:
%load_ext autoreload
%autoreload 2

## Описание эксперимента

Тестировалось усреднение показателей за N шагов и два варианта функции награды: относительная и абсолютная. Итого 3 эксперимента. Также работала система повышения минимального значения напряжения.

## Результаты

In [ ]:
from pathlib import Path

EXPERIMENT_DATE = "2025-09-24"
EXPERIMENTS = {
    "relative_skip1": {
        "time": "18-31-57",
    },
    "relative_skip50": {
        "time": "18-52-01",
    },
    "absolute_skip50": {
        "time": "18-14-49",
    },
}

def get_experiment_path(name: str) -> Path:
    exp = EXPERIMENTS[name]
    return Path(f"../experiments/{name}/{EXPERIMENT_DATE}/{exp['time']}")

PATH_TO_EXP_DIR = get_experiment_path(
    "relative_skip50"
)

LOG_TRAIN_DIR = PATH_TO_EXP_DIR / "train_logs"
LOG_ENV_DIR = PATH_TO_EXP_DIR / "env_logs"

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_tensorboard_df(ax, df, tags=None, title="Training curves"):
    for tag in tags:
        sns.lineplot(data=df, x="step", y=tag, ax=ax, label=tag)
    
    ax.set_title(title, fontsize=14)
    ax.set_xlabel("Step")
    ax.set_ylabel("Value")
    ax.legend(title="Metric")

In [ ]:
from nn_laser_stabilizer.utils import tensorboard_to_df

train_df = tensorboard_to_df(str(LOG_TRAIN_DIR))

Tensorboard содержит не все данные: количество step значительно количества точек.

In [ ]:
print(train_df.info())
print(train_df["step"])

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plot_tensorboard_df(ax, train_df, tags=["Loss/Critic", "Loss/Actor"], title="Training curves")
plt.tight_layout()
plt.savefig(LOG_TRAIN_DIR / "train_logs.pdf")

In [ ]:
import pandas as pd
import re

def parse_pid_log(file_path: str) -> pd.DataFrame:
    pattern = re.compile(
        r"step=(\d+)\s+time=([-+]?\d*\.\d+|\d+)\s+kp=([-+]?\d*\.\d+|\d+)\s+"
        r"ki=([-+]?\d*\.\d+|\d+)\s+kd=([-+]?\d*\.\d+|\d+)\s+"
        r"x=([-+]?\d*\.\d+|\d+)\s+control_output=([-+]?\d*\.\d+|\d+)\s+"
        r"setpoint=([-+]?\d*\.\d+|\d+)"
    )

    data = []
    with open(file_path, "r") as f:
        for line in f:
            match = pattern.search(line)
            if match:
                data.append([
                    int(match.group(1)),      # step
                    float(match.group(2)),    # time
                    float(match.group(3)),    # kp
                    float(match.group(4)),    # ki
                    float(match.group(5)),    # kd
                    float(match.group(6)),    # x
                    float(match.group(7)),    # control_output
                    float(match.group(8))     # setpoint
                ])

    df = pd.DataFrame(data, columns=[
        "step", "time", "Action/kp", "Action/ki", "Action/kd", 
        "Observation/x", "Observation/control_output", "Observation/setpoint"
    ])
    df["reward"] = -abs(df["Observation/setpoint"] - df["Observation/x"]) + 1
    return df

In [ ]:
env_df = parse_pid_log(str(LOG_ENV_DIR / "log.txt"))

In [ ]:
from nn_laser_stabilizer.envs.normalization import denormalize_adc, denormalize_dac

# Делаем обратное преобразование для графика
env_df["Observation/x"] = env_df["Observation/x"].apply(denormalize_adc)
env_df["Observation/setpoint"] = env_df["Observation/setpoint"].apply(denormalize_adc)

env_df["Observation/control_output"] = env_df["Observation/control_output"].apply(denormalize_dac)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plot_tensorboard_df(ax, env_df, tags=["Observation/x", "Observation/setpoint", "Observation/control_output"], title="Observation curves")
plt.tight_layout()
plt.savefig(LOG_ENV_DIR / "observation_logs.pdf")

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(10, 12), sharex=True)

tags = [
    "Observation/x",
    "Observation/setpoint",
    "Observation/control_output",
]

titles = [
    "Observation/x",
    "Observation/setpoint",
    "Observation/control_output",
]

for ax, tag, title in zip(axes, tags, titles):
    plot_tensorboard_df(ax, env_df, tags=[tag], title=title)

plt.tight_layout()
plt.savefig(LOG_ENV_DIR / "observation_logs_split.pdf")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.set_xlim(left=0, right=10_000)
plot_tensorboard_df(ax, env_df, tags=["Observation/x", "Observation/setpoint", "Observation/control_output"], title="Observation curves")
ax.set_ylim(500, 2050)
fig.savefig(LOG_ENV_DIR / "observation_logs_steps.pdf")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
plot_tensorboard_df(ax, env_df, tags=["reward"], title="Reward curve")
plt.tight_layout()
plt.savefig(LOG_ENV_DIR / "reward_logs.pdf")

In [ ]:
action_tags = ["Action/kp", "Action/ki", "Action/kd"]

for tag in action_tags:
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_xlim(0, 1000)

    plot_tensorboard_df(ax, env_df, tags=[tag], title=tag)

    ax.axhline(y=0, color="black", linestyle="--")

    plt.tight_layout()
    plt.savefig(LOG_ENV_DIR / f"{tag.replace('/', '_')}_first_steps.pdf")

for tag in action_tags:
    fig, ax = plt.subplots(figsize=(10, 6))

    plot_tensorboard_df(ax, env_df, tags=[tag], title=tag)

    plt.tight_layout()
    plt.savefig(LOG_ENV_DIR / f"{tag.replace('/', '_')}.pdf")

In [ ]:
from datetime import timedelta

env_df["delta_ms"] = env_df["time"].diff() * 1000
print(env_df["delta_ms"].describe())

total_seconds = env_df["time"].iloc[-1] - env_df["time"].iloc[0]

td = timedelta(seconds=total_seconds)
print(f"Полное время: {td}")